import sys
import os
sys.path.append(os.getcwd())  # add current working directory to Python path


In [5]:
import os

In [6]:
%pwd


'/Users/apple/Downloads/Project/Wine-Quality-Prediction-End-to-End-Project/research'

In [7]:
os.chdir("../")

In [8]:
%pwd

'/Users/apple/Downloads/Project/Wine-Quality-Prediction-End-to-End-Project'

In [9]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/AsifIkbal1/Wine-Quality-Prediction-End-to-End-Project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="AsifIkbal1"
os.environ["MLFLOW_TRACKING_PASSWORD"]="5060c52e0077a940410147fccd4b7e04c7363288"

In [10]:
from dataclasses import dataclass
from pathlib import Path


# 📦 মডেল ইভ্যালুয়েশনের জন্য প্রয়োজনীয় কনফিগারেশন স্টোর করার ডাটাক্লাস
@dataclass(frozen=True)  # 🧊 frozen=True মানে এই ক্লাসটি ইমিউটেবল (পরিবর্তন করা যাবে না)
class ModelEvaluationConfig:
    root_dir: Path  # 📁 রুট ডিরেক্টরি যেখানে ইভ্যালুয়েশন সম্পর্কিত সবকিছু থাকবে
    test_data_path: Path  # 🧪 টেস্ট ডেটার লোকেশন
    model_path: Path  # 🤖 ট্রেইন করা মডেলের লোকেশন
    all_params: dict  # ⚙️ সব হাইপারপ্যারামিটার সংরক্ষণ করার জন্য
    metric_file_name: Path  # 📊 মেট্রিক ফাইল যেখানে মডেলের স্কোর থাকবে
    target_column: str  # 🎯 টার্গেট কলামের নাম
    mlflow_uri: str  # 🔗 MLflow সার্ভারের URI যেখানে ট্র্যাকিং হবে


In [11]:
import sys
import os

# 🔁 current working directory-কে path-এ append করো
sys.path.append(os.path.join(os.getcwd(), "src"))


import sys
print(sys.executable)


In [12]:
import sys
print(sys.executable)


/opt/anaconda3/envs/mlproject1/bin/python


In [13]:

from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories, save_json


In [14]:

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        # 📖 YAML ফাইলগুলো থেকে কনফিগারেশন, প্যারাম এবং স্কিমা লোড করা হচ্ছে
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        # 📁 মূল আর্টিফ্যাক্ট ডিরেক্টরি তৈরি করা হচ্ছে (যদি না থাকে)
        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        # 🔧 কনফিগারেশন, প্যারামিটার এবং টার্গেট কলাম লোড করা হচ্ছে
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        # 📂 মডেল ইভ্যালুয়েশনের জন্য রুট ডিরেক্টরি তৈরি
        create_directories([config.root_dir])

        # ✅ ModelEvaluationConfig অবজেক্ট তৈরি
        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,  # 📁 মূল ডিরেক্টরি
            test_data_path=config.test_data_path,  # 🧪 টেস্ট ডেটার পাথ
            model_path=config.model_path,  # 🤖 ট্রেইনকৃত মডেল ফাইল পাথ
            all_params=params,  # ⚙️ মডেল হাইপারপ্যারামিটারস
            metric_file_name=config.metric_file_name,  # 📊 মেট্রিক সংরক্ষণের জন্য ফাইল পাথ
            target_column=schema.name,  # 🎯 টার্গেট কলামের নাম
            mlflow_uri="https://dagshub.com/AsifIkbal1/Wine-Quality-Prediction-End-to-End-Project.mlflow",  # 🌐 MLflow tracking URI
        )

        return model_evaluation_config  # 🔄 কনফিগারেশন রিটার্ন করা হচ্ছে


In [15]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [16]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        # 🛠️ কনফিগারেশন অবজেক্ট স্টোর করা হচ্ছে
        self.config = config

    def eval_metrics(self, actual, pred):
        # 📐 মডেলের পারফরমেন্স মেট্রিকস হিসেব করা হচ্ছে
        rmse = np.sqrt(mean_squared_error(actual, pred))  # ✅ Root Mean Squared Error
        mae = mean_absolute_error(actual, pred)           # ✅ Mean Absolute Error
        r2 = r2_score(actual, pred)                        # ✅ R² স্কোর
        return rmse, mae, r2  # 🔁 তিনটি মেট্রিক রিটার্ন করা হচ্ছে

    def log_into_mlflow(self):
        # 🧪 টেস্ট ডেটা লোড করা হচ্ছে
        test_data = pd.read_csv(self.config.test_data_path)

        # 🤖 ট্রেইন করা মডেল লোড করা হচ্ছে
        model = joblib.load(self.config.model_path)

        # 🎯 টার্গেট কলাম বাদ দিয়ে X এবং Y আলাদা করা হচ্ছে
        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        # 🌐 MLflow ট্র্যাকিং URI সেট করা হচ্ছে
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        # 🚀 MLflow রান শুরু
        with mlflow.start_run():
            # 📊 প্রেডিকশন করা হচ্ছে
            predicted_qualities = model.predict(test_x)

            # 📏 মেট্রিকস ক্যালকুলেট করা হচ্ছে
            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)

            # 💾 লোকালি মেট্রিকস ফাইল হিসেবে সংরক্ষণ
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            # 📋 MLflow-এ প্যারাম ও মেট্রিক লগ করা হচ্ছে
            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)

            # 🏷️ ফাইল স্টোর না হলে মডেল রেজিস্ট্রি ব্যবহার করা হবে
            if tracking_url_type_store != "file":
                # 📚 মডেল রেজিস্ট্রি-তে মডেল সংরক্ষণ
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                # 📁 শুধুমাত্র মডেল লগ করা হচ্ছে
                mlflow.sklearn.log_model(model, "model")


In [18]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2025-05-18 00:10:22,887: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-05-18 00:10:22,912: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-18 00:10:22,946: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-05-18 00:10:22,949: INFO: common: created directory at: artifacts]
[2025-05-18 00:10:22,951: INFO: common: created directory at: artifacts/model_evaluation]


FileNotFoundError: [Errno 2] No such file or directory: 'artifacts/data_transformation/test.csv'